## Final file for Variable Engineering:
#### All group member code/variable ideas consolidated into one place

In [1]:
## Importing libraries
import pandas as pd
import numpy as np

## Reading the data set
train = pd.read_csv('/Users/EvanCallaghan/Documents/Courses/Predictive Analytics/DATA-448-Project/Data/diabetes_train.csv')
test = pd.read_csv('/Users/EvanCallaghan/Documents/Courses/Predictive Analytics/DATA-448-Project/Data/diabetes_test.csv')

## Printing the first five observations
train.head()

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,Diabetes_012
0,0.0,1.0,1.0,30.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,2.0,30.0,5.0,0.0,0.0,6.0,6.0,8.0,2.0
1,1.0,1.0,1.0,30.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,2.0,0.0,0.0,0.0,1.0,11.0,4.0,7.0,0.0
2,0.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,2.0,0.0,0.0,0.0,1.0,8.0,6.0,7.0,0.0
3,1.0,1.0,1.0,24.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,3.0,0.0,4.0,0.0,1.0,7.0,5.0,7.0,0.0
4,0.0,0.0,1.0,35.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,4.0,0.0,0.0,0.0,1.0,7.0,4.0,1.0,0.0


In [2]:
## All engineered features:

## -----------------------------------------------------------------

## BMI Categoricals
train['BMI_Underweight'] = np.where(train['BMI'] < 18.5, 1, 0)
train['BMI_Healthy'] = np.where((train['BMI'] >= 18.5) & (train['BMI'] < 25), 1, 0)
train['BMI_Overweight'] = np.where((train['BMI'] >= 25) & (train['BMI'] < 30), 1, 0)
train['BMI_Obese'] = np.where(train['BMI'] >= 30, 1, 0)


## Log(BMI)
train['Log_BMI'] = np.log(train['BMI'])

## Creating dummy variables for Sex, Education, and Income
train = pd.concat([train.drop(columns = ['Sex']), pd.get_dummies(train['Sex'])], axis = 1)
train = train.rename(columns = { 0: 'Female', 1: 'Male'})

train = pd.concat([train.drop(columns = ['Education']), pd.get_dummies(train['Education'])], axis = 1)
train = train.rename(columns = { 0: 'Never_Attended', 1: 'Grades_1_8', 3: 'Grades_9_11', 4: 'GED', 5: 'College_1_3', 
                              6: 'College_4+'})

train = pd.concat([train.drop(columns = ['Income']), pd.get_dummies(train['Income'])], axis = 1)
train = train.rename(columns = { 0: '<10,000', 1: '<15,000', 3: '<20,000', 4: '<25,000', 5: '<35,000', 
                                      6: '<50,000',  7: '<75,000',  6: '75,000+'})

## Fruits and Veggies
train['Fruits+Veggies'] = np.where((train['Fruits'] == 1) & (train['Veggies'] == 1), 1, 0)

## Health care issues
train['HealthCareIssues'] = np.where((train['AnyHealthcare'] == 0) & (train['NoDocbcCost'] == 0), 1, 0)

## Poor diet
train['PoorDiet'] = np.where((train['Fruits'] == 0) & (train['Veggies'] == 0) & 
                                (train['HvyAlcoholConsump'] == 1), 1, 0)

## Health categoricals
train['MentHlth_cat'] = np.where((train.MentHlth <=10), 0, 
                                 np.where((train.MentHlth > 10) & (train.MentHlth <= 20), 1, 2))

train['PhysHlth_cat'] = np.where((train.PhysHlth <=10), 0, 
                              np.where((train.PhysHlth > 10) & (train.PhysHlth <= 20), 1, 2))

train['GenHlth_cat'] = np.where((train.GenHlth <=2), 0, 
                             np.where((train.GenHlth > 3) & (train.GenHlth <= 5), 1, 2))

## Creating interactions
train['Interaction_1'] = train['HighBP'] * train['GenHlth']
train['Interaction_2'] = train['HighBP'] * train['GenHlth_cat']
train['Interaction_3'] = train['HighBP'] * train['HighChol']
train['Interaction_4'] = train['GenHlth'] * train['GenHlth_cat']
train['Interaction_5'] = train['GenHlth'] * train['HighChol']
train['Interaction_6'] = train['GenHlth_cat'] * train['HighChol']

## Creating tree interactions
train['Tree_1'] = np.where((train['Interaction_2'] <= 0.5) & (train['Interaction_5'] <= 1.5) & (train['Age'] <= 8.5), 1, 0)
train['Tree_2'] = np.where((train['Interaction_2'] <= 0.5) & (train['Interaction_5'] <= 1.5) & (train['Age'] > 8.5), 1, 0)
train['Tree_3'] = np.where((train['Interaction_2'] <= 0.5) & (train['Interaction_5'] > 1.5) & (train['Log_BMI'] <= 3.384), 1, 0)
train['Tree_4'] = np.where((train['Interaction_2'] <= 0.5) & (train['Interaction_5'] > 1.5) & (train['Log_BMI'] > 3.384), 1, 0)
train['Tree_5'] = np.where((train['Interaction_2'] > 0.5) & (train['Interaction_5'] <= 3.5) & (train['BMI'] <= 30.5), 1, 0)
train['Tree_6'] = np.where((train['Interaction_2'] > 0.5) & (train['Interaction_5'] <= 3.5) & (train['BMI'] > 30.5), 1, 0)
train['Tree_7'] = np.where((train['Interaction_2'] > 0.5) & (train['Interaction_5'] > 3.5) & (train['Log_BMI'] <= 3.481), 1, 0)
train['Tree_8'] = np.where((train['Interaction_2'] > 0.5) & (train['Interaction_5'] > 3.5) & (train['Log_BMI'] > 3.481), 1, 0)

## *Note that these are only on the train data-frame*